**извлечение признаков из текста на естественном языке**

word2vec : skip-gram


Евгений Борисов borisov.e@solarl.ru

In [1]:
# import sys
import re
import gzip
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# # загружаем предварительно очищенный текст
# with gzip.open('../data/text.txt.gz','rt',encoding='utf-8') as f: text = f.read()

In [3]:
# загружаем текст
with gzip.open('../data/dostoevsky-besy-p3.txt.gz','rt',encoding='utf-8') as f: 
    text = [ s.strip() for s in re.split(r'[\n.?!]',f.read()) if s.strip() ]

In [4]:
len(text)

5708

In [5]:
# убираем заголовок
text = text[7:]

---

In [6]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

In [7]:
# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е',] # замена ё для унификации
       ,[r'[^а-я ]+',' '] # оставляем только буквы и пробел
       ,[r'^- *',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

In [8]:
text = list(map(string_normalizer,text))

In [9]:
text[:5]

['праздник состоялся несмотря ни на какие недоумения прошедшего шпигулинского дня',
 'я думаю что если бы даже лембке умер в ту самую ночь то праздник все таки бы состоялся на утро до того много соединяла с ним какого то особенного значения юлия михайловна',
 'увы она до последней минуты находилась в ослеплении и не понимала настроения общества',
 'никто под конец не верил что торжественный день пройдет без какого нибудь колоссального приключения без развязки как выражались иные заранее потирая руки',
 'многие правда старались принять самый нахмуренный и политический вид но вообще говоря непомерно веселит русского человека всякая общественная скандальная суматоха']

In [10]:
# разрезаем текст на слова
text = [ [ w.strip() for w in s.split() if w.strip() ] for s in text ]

In [11]:
# text[:2]

In [12]:
context_deep = 2 # глубина контекста

In [13]:
# удаляем короткие словосочетания
text = [ s for s in text if len(s) > context_deep*2 ]
len(text)

4223

In [14]:
words = sorted(set(sum(text,[]))) # словарь из текста
words

['а',
 'авдотьи',
 'авось',
 'авторских',
 'агатовым',
 'агафьи',
 'ада',
 'адвокаты',
 'административным',
 'администрации',
 'азарта',
 'азартом',
 'ай',
 'аккуратно',
 'аккуратностью',
 'аккуратный',
 'акушерка',
 'акушерки',
 'акушеркой',
 'ал',
 'алексей',
 'алексею',
 'алексея',
 'али',
 'аллегории',
 'аллегорию',
 'аль',
 'альбоме',
 'амбициозной',
 'америке',
 'америку',
 'аминь',
 'амфилады',
 'анализируем',
 'ангел',
 'ангела',
 'ангелу',
 'ангелы',
 'ангельский',
 'ангельском',
 'англичан',
 'англичанина',
 'андрее',
 'андреем',
 'андрей',
 'андрею',
 'андрея',
 'анекдотами',
 'анекдоты',
 'анисим',
 'анисима',
 'анисиме',
 'анисимова',
 'анк',
 'анонимное',
 'антитез',
 'антонович',
 'антоновича',
 'антоновиче',
 'антоновичем',
 'антоновичу',
 'анфилады',
 'аплодировавшая',
 'аплодировала',
 'аплодировали',
 'аплодировать',
 'аплодисман',
 'аплодисмент',
 'аплодисмента',
 'аплодисментом',
 'апокалипсиса',
 'аптекарь',
 'аптекаря',
 'аптеку',
 'аргументом',
 'арене',
 'арест

In [15]:
# заменяем слова в тексте их номерами в словаре
vocab =  { w:i for i,w in enumerate(words) }
code = [ [ vocab[w] for w in s ] for s in text ]

In [16]:
# code

In [17]:
n_words = len(words) # количество слов в словаре
# m = len(sum(code,[])) # количество слов в тексте 
print( "размер словаря: %i слов" % n_words )
# print( "размер текста: %i слов" % m )

размер словаря: 13151 слов


In [18]:
# # унитарное кодирование словаря (one-hot-encoding)
# Ve = np.eye( n ) 

# # кодированный текст (последовательность кодов слов)
# Te = Ve[code,:] 

In [19]:
# Te

In [20]:
# code

In [21]:
# [ контекст, слово ]
def pack_history(s,cd=context_deep):
    return [ [ s[i-cd:i], s[i], s[i+1:i+cd+1] ] for i in range(cd,len(s)-cd) ] 

In [22]:
context = sum( [ pack_history(s,cd=context_deep) for s in code ], [] )

In [23]:
W = [ c[1] for c in context ] # коды слов
C = [ c[0]+c[2] for c in context ] # коды контекста слов

In [ ]:
# # выполняем OHE номеров символов 
# def encode_abc_bin(codes,abc_len=len(abc)):
#     return np.array([ np.eye(abc_len,dtype=int)[c] for c in codes ], dtype=int)[np.newaxis,:]


In [ ]:
# # унитарное кодирование словаря (one-hot-encoding)


# np.array([ np.eye(n_words,dtype=int)[c] for c in W ], dtype=int)[np.newaxis,:]

# np.eye( n_words )[3]

# [ np.eye( n_words )[c] for c in W ]

In [ ]:
# # формируем учебный набор...

# c = 2  # размер окна контекста, количество слов перед текущим словом и после него
# W = [] # слова
# C = [] # контекст для слов

# for i in range(c,m-c) :
#     W.append( Te[i,:] ) # код текущего слова
#     # контекст для текущего слова
#     C.append( np.vstack( ( Te[i-c:i,:] , Te[i+1:i+c+1,:] ) ).reshape(1,2*c,n) )

In [ ]:
# W = np.vstack(W)
# W.shape

In [ ]:
# C = np.concatenate(C)
# C.shape

In [ ]:
# np.savez( "result/data.npz", W=W, C=C )
# scipy.io.savemat("result/data.mat", dict(W=W, C=C))

---

In [ ]:
# ф-ция активации скрытого слоя - линейная
# def act(s): return s

# ф-ция активации выходного слоя
def softmax(s): 
    e = np.exp(s)
    return e/e.sum(axis=1).reshape(s.shape[0],1)

In [ ]:
def w2v_step(W,Vi,Vo):
    H = W.dot(Vi) # значения скрытого слоя
    U = H.dot(Vo) # состояния выходного слоя
    O = softmax(U) # выход сети
    return H,U,O


Функция потери

$$E = \left| \sum\limits_i\log\left( \sum\limits_k \exp(U_{ik}) \right) - \sum\limits_i\sum\limits_k\sum\limits_j (U_{ik} * Q_{ijk}) \right| $$


$U_{ik}$ состояние k-того нейрона выходного слоя для слова $i$     
$Q_{ij}$ слово $j$ контекста слова $i$   
где ∗ - операция поэлементного умножения векторов

In [ ]:
def w2v_loss(U,C):
    n,cws,v = C.shape 
    # количество примеров
    # размер окна контекста   
    # количество слов в словаре

    Us = np.log( np.exp(U).sum(axis=1) ).sum()

    Uo = 0.0
    for i in range(cws): # для всех слов контекста
        Ci = C[:,i,:].reshape([n,v]) # набор слов контекста i
        Uo += (U*Ci).sum() # значения выходного слоя для слов x контекста i

    return np.abs(Us-Uo)/n

In [ ]:
def w2v_grad(C,W,H,O,Vo):
    n,cws,v = C.shape
        # количество примеров
        # размер окна контекста   
        # количество слов в словаре

    gVi = gVo = 0.0 

    for i in range(cws):
        Ci = C[:,i,:].reshape([n,v]) # слово i контекста
        D = O - Ci # ошибка на слове контекста i
        gVo += D.T.dot(H).T
        gVi += W.T.dot(D).dot(Vo.T)

    return gVi,gVo

---

In [ ]:
def w2v_weigth_norm(v,w):
    nn = np.linalg.norm( np.hstack([ v.flatten(), w.flatten() ]) )
    #nn = np.abs( np.hstack([ v.flatten(), w.flatten() ]) ).max()
    return (v/nn,w/nn) if nn!=0.0 else (v,w)

---

In [ ]:
n = W.shape[1] # количество слов в словаре
k = 30 # размер скрытого слоя H

Vi = np.random.normal(scale=0.01,size=(n,k))
Vo = np.random.normal(scale=0.01,size=(k,n))

In [ ]:
a = 0.91 # скорость обучения

er = [1e10]

for i in range(100):
    H,U,O = w2v_step(W,Vi,Vo) # вычисляем состояния слоёв и выход
    er.append(w2v_loss(U,C)) # вычисляем ошибку
    print(i+1, er[-1])
    if er[-1]>er[-2]: 
        Vi, Vo = Vi_old, Vo_old
        break
        
    gVi, gVo = w2v_grad(C,W,H,O,Vo) # вычисляем градиент ф-ции потери
    gVi,gVo = w2v_weigth_norm(gVi,gVo) # нормируем значения градиента
    
    Vi_old, Vo_old =  Vi, Vo # сохраняем старые веса
    
    Vi,Vo = Vi-a*gVi, Vo-a*gVo # корректируем веса


In [ ]:
er=er[1:]
plt.plot(er)
plt.grid()
plt.show()

---

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
D = pairwise_distances(Vi) # матрица расстояний
R = np.argsort(D) # номера слов в порядке увеличения расстояния

In [ ]:
vocab = { i:w for i,w in enumerate(words) } # нумеруем слов в словаре

nn = np.random.permutation(len(words))[:10] # выбираем случайно N слов

# для выбранных слов печатаем близкие по w2v
{ vocab[i] : [ vocab[j] for j in  R[i,1:3]  ] for i in nn }